Due to BRENDA's license, you have to download the database as a text file after accepting usage conditions [here](https://www.brenda-enzymes.org/download.php).

# Load data

In [1]:
from brendapyrser import BRENDA
dataFile = 'data/brenda_2023_1.txt'
brenda = BRENDA(dataFile)

# Create Reaction Database

In [3]:
reaction_ec_numbers = []
reaction_ec_annotation = []
reaction_names = []
reaction_systematic_name = []
reaction_str = []
reaction_type = []
reaction_substrates = []
reaction_products = []
data = []

In [11]:
from utils import extract_substrates,extract_products, extract_just_ec_number, extract_ec_annotation
import json
for r in brenda.reactions:
    reaction_ec_numbers.append(extract_just_ec_number(r.ec_number))
    reaction_ec_annotation.append(extract_ec_annotation(r.ec_number))
    reaction_names.append(r.name)
    reaction_systematic_name.append(r.systematic_name)
    reaction_str.append(r.reaction_str)
    reaction_type.append(r.reaction_type)
    reaction_substrates.append(extract_substrates(r.substratesAndProducts))
    reaction_products.append(extract_products(r.substratesAndProducts))
    item = {
        'ec_number': extract_just_ec_number(r.ec_number),
        'ec_annotation': extract_ec_annotation(r.ec_number),
        'name': r.name,
        'systematic_name': r.systematic_name,
        'str': r.reaction_str,
        'type': r.reaction_type,
        'substrates': json.dumps(extract_substrates(r.substratesAndProducts)),
        'products': json.dumps(extract_products(r.substratesAndProducts))
    }
    for key in item.keys():
        if not item[key]:
            item[key] = None
    if item['ec_number']:
        data.append(item)

In [5]:
# with mysql running
import pymysql
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '123456',
    'charset':'utf8mb4'
}
try:
    connection = pymysql.connect(**db_config)
    cursor = connection.cursor()
except pymysql.Error as e:
    print("Database connection error:", e)
try:
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS Reaction")
    cursor.execute(f"USE Reaction")
    cursor.execute("""
            CREATE TABLE IF NOT EXISTS reactions (
                ec_number VARCHAR(20) PRIMARY KEY, 
                ec_annotation TEXT,
                name VARCHAR(100),
                systematic_name VARCHAR(100),
                str TEXT,
                type VARCHAR(100),
                substrates JSON,
                products JSON
            ) CHARACTER SET utf8mb4;""")
    for item in data:
        try:
            insert_query = f"INSERT INTO reactions ({', '.join(item.keys())}) VALUES ({', '.join(['%s'] * len(item))})"
            cursor.execute(insert_query, tuple(item.values()))
        except pymysql.Error as e:
            print(e)
    connection.commit()
    query = f"SELECT * FROM reactions WHERE ec_number = %s"
    cursor.execute(query, ('1.1.1.1'))
    rows = cursor.fetchall()
    cursor.close()
    connection.close()
except pymysql.Error as e:
    print(e)

In [14]:
try:
    connection = pymysql.connect(**db_config)
    cursor = connection.cursor()
except pymysql.Error as e:
    print("Database connection error:", e)
try:
    cursor.execute(f"USE Reaction")
    query = f"SELECT * FROM reactions WHERE ec_number = %s"
    cursor.execute(query, ('1.1.1.109'))
    rows = cursor.fetchall()
except pymysql.Error as e:
    print(e)

# Create Enzyme Database

In [ ]:
reaction_ec_numbers = []
reaction_species = []
reaction_reference = []
reaction_kcat = []
reaction_km = []
reaction_specific_activities = []
data = []

In [31]:
from utils import parse_kcat,parse_km
for r in brenda.reactions:
    reaction_ec_numbers.append(extract_just_ec_number(r.ec_number))
    reaction_species.append(r.getSpeciesDict())
    reaction_reference.append(r.getReferencesDict())
    reaction_kcat.append(parse_kcat(r))
    reaction_km.append(parse_km(r))
    reaction_specific_activities.append()
    

'query = "SELECT ec_number FROM reactions"\ncursor.execute(query)\ncolumn_data = cursor.fetchall()\ncursor.close()\nconnection.close()'

In [27]:
brenda.reactions[1].proteins

{'1': {'name': 'Gallus gallus', 'proteinID': '', 'refs': ['44']},
 '2': {'name': 'Cricetulus griseus', 'proteinID': '', 'refs': ['1']},
 '3': {'name': 'Vicia faba', 'proteinID': '', 'refs': ['4']},
 '4': {'name': 'Drosophila melanogaster',
  'proteinID': '',
  'refs': ['8', '43', '58', '62', '63', '64', '65']},
 '5': {'name': 'Bacillus subtilis', 'proteinID': '', 'refs': ['304']},
 '6': {'name': 'Mus musculus',
  'proteinID': '',
  'refs': ['9', '48', '117', '119', '141', '142', '200', '212']},
 '7': {'name': 'Thermus thermophilus',
  'proteinID': '',
  'refs': ['169', '268']},
 '8': {'name': 'Escherichia coli', 'proteinID': '', 'refs': ['135']},
 '9': {'name': 'Homo sapiens',
  'proteinID': '',
  'refs': ['10',
   '11',
   '12',
   '13',
   '14',
   '15',
   '16',
   '17',
   '18',
   '19',
   '20',
   '21',
   '22',
   '23',
   '53',
   '96',
   '107',
   '109',
   '115',
   '116',
   '119124',
   '150',
   '180',
   '186',
   '191',
   '194',
   '206',
   '227',
   '228',
   '272']}

In [33]:
brenda.reactions[1].getSpeciesDict()

{'1': {'name': 'Gallus gallus', 'proteinID': '', 'refs': ['44']},
 '2': {'name': 'Cricetulus griseus', 'proteinID': '', 'refs': ['1']},
 '3': {'name': 'Vicia faba', 'proteinID': '', 'refs': ['4']},
 '4': {'name': 'Drosophila melanogaster',
  'proteinID': '',
  'refs': ['8', '43', '58', '62', '63', '64', '65']},
 '5': {'name': 'Bacillus subtilis', 'proteinID': '', 'refs': ['304']},
 '6': {'name': 'Mus musculus',
  'proteinID': '',
  'refs': ['9', '48', '117', '119', '141', '142', '200', '212']},
 '7': {'name': 'Thermus thermophilus',
  'proteinID': '',
  'refs': ['169', '268']},
 '8': {'name': 'Escherichia coli', 'proteinID': '', 'refs': ['135']},
 '9': {'name': 'Homo sapiens',
  'proteinID': '',
  'refs': ['10',
   '11',
   '12',
   '13',
   '14',
   '15',
   '16',
   '17',
   '18',
   '19',
   '20',
   '21',
   '22',
   '23',
   '53',
   '96',
   '107',
   '109',
   '115',
   '116',
   '119124',
   '150',
   '180',
   '186',
   '191',
   '194',
   '206',
   '227',
   '228',
   '272']}

In [6]:
brenda.reactions[1].KMvalues

{'benzyl alcohol': [{'value': 0.05,
   'species': ['Rattus norvegicus'],
   'meta': '#10# isoenzyme ADH-3, pH 10.0 <49>',
   'refs': [' Julia, P.; Farres, J.; Pares, X.: Characterization of threeisoenzymes of rat alcohol dehydrogenase. Tissue distribution andphysical and enzymatic properties. Eur. J. Biochem. (1987) 162,179-189. {Pubmed:3816781} (c)']},
  {'value': 1.4,
   'species': ['Rattus norvegicus'],
   'meta': '#10# isoenzyme ADH-1, pH 10.0 <49>',
   'refs': [' Julia, P.; Farres, J.; Pares, X.: Characterization of threeisoenzymes of rat alcohol dehydrogenase. Tissue distribution andphysical and enzymatic properties. Eur. J. Biochem. (1987) 162,179-189. {Pubmed:3816781} (c)']},
  {'value': 0.12,
   'species': ['Saccharolobus solfataricus'],
   'meta': '#109# mutant enzyme W95L, in 0.1 Mglycine-NaOH buffer (pH 10.5), at 65°C <207>',
   'refs': [' Pennacchio, A.; Esposito, L.; Zagari, A.; Rossi, M.; Raia, C.A.:Role of Tryptophan 95 in substrate specificity and structural stabilityo

In [32]:
brenda.reactions[1].mechanism

[{'species': ['4', '43'],
  'refs': ['31', '43'],
  'meta': 'orderedbi-bi mechanism'},
 {'species': ['4', '78'],
  'refs': ['63'],
  'meta': 'rapid equilibrium random mechanism'},
 {'species': ['9'],
  'refs': ['23'],
  'meta': 'ordered bi bi mechanism with cofactor adding first to form abinary enzyme complex'},
 {'species': ['43'],
  'refs': ['35'],
  'meta': 'isoenzyme EE and SS: ordered bi bimechanism'},
 {'species': ['11', '35'],
  'refs': ['91'],
  'meta': 'mechanism is predominantly ordered withethanol, but partially random with butanol'},
 {'species': ['43'],
  'refs': ['41'],
  'meta': 'kinetic mechanismis random for ethanol oxidation and compulsory ordered for acetaldehydereduction'},
 {'species': ['40'],
  'refs': ['85'],
  'meta': 'oxidizes ethanol in an ordered bi-bi mechanismwith NAD+ as the first substrate fixed'},
 {'species': ['11'],
  'refs': ['90'],
  'meta': 'compulsory-ordermechanism with the rate-limiting step being the dissociation of theproduct enzyme-NAD+ comple

In [23]:
len(brenda.getOrganisms())

12514